In [224]:
from enum import Enum
from heapq import heappush

import numpy as np
from sortedcontainers import SortedList


class EventType(Enum):
    BEGIN = 1
    END = 2
    INTERSECT = 3

In [225]:
def generate_random_segments(n=10, min_intersections=2, coord_range=(0, 10)):
    # Создаем массив для хранения отрезков: (n, 2, 2), где каждый отрезок описывается двумя точками (x1,y1)-(x2,y2)
    segments = np.zeros((n, 2, 2))

    # Генерируем min_intersections пар пересекающихся отрезков
    for i in range(0, min_intersections * 2, 2):
        # Первый отрезок (горизонтальный или вертикальный)
        if np.random.choice([True, False]):
            # Горизонтальный отрезок
            y = np.random.uniform(*coord_range)
            x1, x2 = np.random.uniform(*coord_range, size=2)
            segments[i] = np.array([[x1, y], [x2, y]])
        else:
            # Вертикальный отрезок
            x = np.random.uniform(*coord_range)
            y1, y2 = np.random.uniform(*coord_range, size=2)
            segments[i] = np.array([[x, y1], [x, y2]])

        # Второй отрезок, гарантированно пересекающий первый
        if segments[i, 0, 0] == segments[i, 1, 0]:  # Первый вертикальный → второй горизонтальный
            x = segments[i, 0, 0]
            y = np.random.uniform(*coord_range)
            segments[i + 1] = np.array([[x - 2, y], [x + 2, y]])  # Пересекает первый
        else:  # Первый горизонтальный → второй вертикальный
            y = segments[i, 0, 1]
            x = np.random.uniform(*coord_range)
            segments[i + 1] = np.array([[x, y - 2], [x, y + 2]])  # Пересекает первый

    # Остальные отрезки — случайные
    for i in range(min_intersections * 2, n):
        segments[i] = np.random.uniform(*coord_range, size=(2, 2))

    return segments


In [226]:
def point_on_segment(a, b, s):
    ax, ay = a
    bx, by = b
    cx, cy = s

    cross_product = (bx - ax) * (cy - ay) - (by - ay) * (cx - ax)
    if abs(cross_product) > 1e-12:  # учитываем погрешность вычислений
        return False

    in_x = min(ax, bx) <= cx <= max(ax, bx)
    in_y = min(ay, by) <= cy <= max(ay, by)

    return in_x and in_y

In [227]:
class Event:
    def __init__(self, type, x, y, related_segments=None):
        if related_segments is None:
            self.related_segments = set()
        else:
            self.related_segments = related_segments
        self.y = y
        self.x = x
        self.type = type

    type: EventType

    def __str__(self):
        return f"{self.type.name} x: {self.x} y: {self.y}"

    def __repr__(self):
        return self.__str__()

    def __lt__(self, other):
        return self.y > other.y or (self.y == other.y and (
                self.x < other.x or self.x == other.x and (
                self.type == EventType.BEGIN and other.type == EventType.END or self.type == EventType.BEGIN and other.type == EventType.INTERSECT or
                self.type == EventType.INTERSECT and other.type == EventType.END) and other.type != self.type))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.type == other.type

    def __ne__(self, other):
        return not self.__eq__(other)


In [228]:
def find_segment_x_by_current_y(seg, y):
    return seg.start[0] + (seg.start[1] - seg.start[0]) * (y - seg.start[0]) / (seg.start[1] - seg.start[0])

In [229]:
class Segment:
    def __init__(self, label, nparray):
        self.label = label
        if (nparray[0, 1] > nparray[1, 1]) or ((nparray[0, 1] == nparray[1, 1]) and (nparray[0, 0] > nparray[1, 0])):
            self.start = nparray[0]
            self.end = nparray[1]
        else:
            self.start = nparray[1]
            self.end = nparray[0]
        self.current_x = self.start[0]
        self.current_y = self.start[1]

    start: np.ndarray
    end: np.ndarray

    def __lt__(self, other):
        return self.current_x < other.current_x or (self.current_x == other.current_x and self.end[0] < other.end[0])

    def __eq__(self, other):
        return self.label == other.label

    def __neq__(self, other):
        return self.label != other.label

    def __hash__(self):
        return self.label



In [230]:
def handle_event_point(p, events_q, seg_status, labels_set):
    start_in_p = p.related_segments
    end_in_p = set()
    contain_p = set()
    

def find_intersections(segments_array):
    events_q = SortedList()
    seg_status = SortedList()
    segments = list()
    labels_set = set()
    for i, s in enumerate(segments_array):
        seg = Segment(i, s)
        segments.append(seg)
        beginning = Event(EventType.BEGIN, seg.start[0], seg.start[1])
        end = Event(EventType.END, seg.end[0], seg.end[1])
        if not beginning in events_q:
            events_q.add(beginning)
        if not end in events_q:
            events_q.add(end)
    for s in segments:
        start = s.start
        beg_e = Event(EventType.BEGIN, start[0], start[1])
        end_e = Event(EventType.END, start[0], start[1])
        if beg_e in events_q:
            events_q[events_q.index(beg_e)].related_segments.add(s)
        if end_e in events_q:
            events_q[events_q.index(end_e)].related_segments.add(s)

    while len(events_q) != 0:
        p = events_q.pop(0)
        handle_event_point(p, events_q, seg_status, labels_set)

segments = generate_random_segments(n=10, min_intersections=2)

intersections = find_intersections(segments)
print(intersections)